In [3]:
import platform
platform.python_implementation()

'CPython'

In [190]:
%run 'qboot.py'

### Numba Test

In [191]:
pauli(3).commutate(pauli(1))

2i Y0

In [192]:
maj(0) @ maj(1)

χ0 χ1

In [188]:
from numba import njit
from numba.typed import List
class A:
    def do(self):
        return self.term_mul(List(((0,1),)),List(((0,2),)))
    
    @staticmethod
    @njit
    def term_mul(term1, term2):
        ''' redefine term-level multiplication rule '''
        if len(term1) == 0: # term1 is identity operator
            return term2, 1
        if len(term2) == 0: # term2 is identity operator
            return term1, 1
        n1 = len(term1) # length of term1
        n2 = len(term2) # length of term2
        i1 = 0 # term1 pointer
        i2 = 0 # term2 pointer
        term = [(0,0)][:0] # to collect indices in the resulting term
        phase = 1 # to track the phase factor
        pauli_rule = [0,1,2,3,1,0,3,2,2,3,0,1,3,2,1,0]
        phase_rule = [1,1,1,1,1,1,1j,-1j,1,-1j,1,1j,1,1j,-1j,1]
        while i1 < n1 and i2 < n2:
            ind1, mu1 = term1[i1]
            ind2, mu2 = term2[i2]
            if ind1 == ind2: # indices collide
                mu12 = 4 * mu1 + mu2
                mu = pauli_rule[mu12]
                # if mu == 0: identity operator ignored
                if mu != 0:
                    phase *= phase_rule[mu12]
                    term.append((ind1, mu))
                i1 += 1
                i2 += 1
            else:
                if ind1 < ind2:
                    term.append((ind1, mu1))
                    i1 += 1
                else: # ind1 > ind2
                    term.append((ind2, mu2))
                    i2 += 1
        for i in range(i1, n1): # if term1 not exhausted
            term.append(term1[i]) # dump the rest
        for i in range(i2, n2): # if term2 not exhausted
            term.append(term2[i]) # dump the rest
        return term, phase

In [189]:
a=A()
a.do()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mCan't unify return type from the following types: Tuple(ListType[UniTuple(int64 x 2)], Literal[int](1)), Tuple(list(UniTuple(int64 x 2))<iv=None>, complex128)
[1mReturn of: IR name '$350return_value.3', type 'Tuple(list(UniTuple(int64 x 2))<iv=None>, complex128)', location: [1m
File "../../../../../../var/folders/_g/zrwr7jps6q75r7vyx6kg_ms40000gn/T/ipykernel_34567/3322329341.py", line 46:[0m
[1m<source missing, REPL/exec in use?>[0m[0m
[1mReturn of: IR name '$20return_value.3', type 'Tuple(ListType[UniTuple(int64 x 2)], Literal[int](1))', location: [1m
File "../../../../../../var/folders/_g/zrwr7jps6q75r7vyx6kg_ms40000gn/T/ipykernel_34567/3322329341.py", line 12:[0m
[1m<source missing, REPL/exec in use?>[0m[0m[0m

In [54]:
A.term_mul((0,),(1,))

([0, 1], 1)

In [94]:
@njit
def empty():
    return (), 1

In [161]:
a,b = numpy.array([1,2])
(a,b)

(1, 2)

In [185]:
from numba.typed import List
List((1,1))

ListType[int64]([1, 1, ...])

### Quantum Bootstrap

Define fermion creation/annihilation/number operators.

In [154]:
def c(i,s):
    return (maj(4*i+2*s) + 1j * maj(4*i+2*s+1))/2
def cd(i,s):
    return (maj(4*i+2*s) - 1j * maj(4*i+2*s+1))/2
def n(i,s):
    return cd(i,s) @ c(i,s)

Construct 1D Hubbard model Hamiltonian.

In [157]:
L = 3
t, U = 1., 1.
H = - t * sum(sum(cd(i,s) @ c((i+1)%L,s)+cd((i+1)%L,s) @ c(i,s) for s in [0,1]) for i in range(L)) \
    + U * sum(n(i,0) @ n(i,1) for i in range(L))
H

ValueError: cannot compute fingerprint of empty list

Perform quantum bootstrap.

In [145]:
rho = qboot(H, n=11, show_progress=True)
rho

     len  weight
 0:    1    0.0
 1:   21    2.1
 2:  144    4.1
 3:  375    6.0
 4:  519    7.8
 5:  543    9.7
 6:  543   11.0
 7:  543    7.6
 8:  543    6.9
 9:  543    6.6
10:  543    5.6
11:  543    5.3
Krylov space constructed.
     pcost       dcost       gap    pres   dres   k/t
 0: -5.3620e-02 -1.5356e+01  1e+01  2e-16  8e+00  1e+00
 1: -3.7664e-01 -3.1636e+00  2e+00  2e-15  1e+00  2e-01
 2: -1.4266e+00 -2.7157e+00  1e+00  1e-15  7e-01  1e-01
 3: -2.5492e+00 -3.0958e+00  7e-01  4e-15  3e-01  5e-02
 4: -2.8435e+00 -3.1361e+00  4e-01  2e-15  2e-01  3e-02
 5: -2.9509e+00 -3.3954e+00  2e+00  2e-14  3e-01  9e-02
 6: -4.1930e+00 -4.2441e+00  4e-01  2e-14  4e-02  3e-02
 7: -4.4479e+00 -4.4486e+00  6e-03  7e-15  6e-04  5e-04
 8: -4.4515e+00 -4.4515e+00  6e-05  1e-14  6e-06  5e-06
 9: -4.4516e+00 -4.4516e+00  6e-07  1e-14  6e-08  5e-08
Optimal solution found.


-0.661i χ1 χ4 +0.661i χ2 χ7 -0.661i χ3 χ6 +0.661i χ4 χ9 +0.661i χ6 χ11 -0.661i χ7 χ10 -0.661i χ1 χ8 -0.661i χ3 χ10 +0.661i χ2 χ11 -0.333i χ2 χ3 -0.333i χ6 χ7 -0.333i χ10 χ11 -0.333i χ8 χ9 -0.0208 χ8 χ9 χ10 χ11 +0.172i χ0 χ1 χ2 χ4 χ5 χ7 -0.172i χ0 χ1 χ3 χ4 χ5 χ6 +0.245i χ0 χ1 χ4 χ5 χ6 χ11 -0.245i χ0 χ1 χ4 χ5 χ7 χ10 -0.245i χ0 χ1 χ3 χ4 χ5 χ10 +0.245i χ0 χ1 χ2 χ4 χ5 χ11 -0.156i χ0 χ1 χ2 χ3 χ4 χ5 -0.156i χ0 χ1 χ4 χ5 χ6 χ7 -0.021i χ0 χ1 χ4 χ5 χ10 χ11 -1i χ0 χ1 χ4 χ5 χ8 χ9 -0.333 χ0 χ1 χ4 χ5 χ8 χ9 χ10 χ11 +0.172i χ0 χ2 χ3 χ5 χ6 χ7 +0.453i χ0 χ2 χ4 χ5 χ7 χ9 -0.453i χ0 χ2 χ5 χ6 χ7 χ11 +0.453i χ0 χ1 χ2 χ5 χ7 χ8 -0.453i χ0 χ2 χ3 χ5 χ7 χ10 -0.422i χ0 χ2 χ5 χ7 χ10 χ11 -0.422i χ0 χ2 χ5 χ7 χ8 χ9 -0.422 χ0 χ2 χ5 χ7 χ8 χ9 χ10 χ11 ...

Evaluate energy expectation value (Ground truth: -3.70156)

In [146]:
rho.inner(H).real

-3.7015615921117098

Timing

In [147]:
%timeit H.krylov(11)

252 ms ± 953 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [148]:
%prun H.krylov(11)

         571286 function calls (571285 primitive calls) in 0.476 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       12    0.283    0.024    0.463    0.039 qboot.py:455(__matmul__)
   106942    0.076    0.000    0.090    0.000 qboot.py:179(__iadd__)
   106942    0.075    0.000    0.075    0.000 qboot.py:547(maj_term_mul)
   107024    0.016    0.000    0.016    0.000 qboot.py:25(__init__)
   120984    0.009    0.000    0.009    0.000 {built-in method builtins.abs}
   107001    0.005    0.000    0.005    0.000 {built-in method builtins.isinstance}
       23    0.004    0.000    0.004    0.000 qboot.py:148(__add__)
       23    0.003    0.000    0.004    0.000 qboot.py:287(inner)
     7800    0.001    0.000    0.001    0.000 {method 'pop' of 'dict' objects}
       23    0.001    0.000    0.001    0.000 qboot.py:111(<dictcomp>)
     4871    0.001    0.000    0.001    0.000 qboot.py:318(<genexpr>)
       23    0.001    0.000 